1. https://ai.plainenglish.io/understanding-low-rank-adaptation-lora-for-efficient-fine-tuning-of-large-language-models-082d223bb6db
2. https://medium.com/@manindersingh120996/practical-guide-to-fine-tune-llms-with-lora-c835a99d7593
3. https://medium.com/@imabhi1216/fine-tuning-a-vision-transformer-vit-model-with-a-custom-dataset-37840e4e9268

# I. Pip Install / Load Libraries / Load Dataset form Hugging face

In [ ]:
!pip install datasets torchvision accelerate evaluate

In [ ]:
import psutil

# Lấy thông tin RAM (số byte khả dụng)
available_ram = psutil.virtual_memory().available
# Lấy số lượng lõi CPU (logical)
cpu_count = psutil.cpu_count(logical=True)

print(f"Available RAM: {available_ram / (1024**3):.2f} GB")
print(f"CPU count: {cpu_count}")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datasets import load_from_disk

from datasets import load_dataset, Features, Value
from transformers import ViTFeatureExtractor
import torch
from torchvision import transforms
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
dataset_dict  = load_dataset("ashraq/fashion-product-images-small")
dataset_dict

# II. Basic Preprocessing and Understanding Data

In [ ]:
train_dataset = dataset_dict["train"]
train_dataset_org = dataset_dict["train"]

In [ ]:
shown_labels = set()
plt.figure(figsize=(15, 10))

# Loop through the dataset and plot the first image of each label
for i, sample in enumerate(train_dataset):
    label = sample["masterCategory"]  # lấy nhãn trực tiếp vì nó đã là chuỗi
    if label not in shown_labels:
        plt.subplot(1, len(set([s["masterCategory"] for s in train_dataset])), len(shown_labels) + 1)
        plt.imshow(sample["image"])
        plt.title(label)
        plt.axis("off")
        shown_labels.add(label)
        # Nếu đã hiển thị hết các nhãn, dừng vòng lặp
        if len(shown_labels) == len(set([s["masterCategory"] for s in train_dataset])):
            break

plt.show()

## 2.1 Add Random Price Column

In [ ]:
def assign_price(masterCategory, subCategory, articleType, year, season, usage):
    master_price_range = {
        'Apparel': (10, 20), 'Accessories': (5, 100), 'Footwear': (20, 200),
        'Personal Care': (5, 300), 'Free Items': (0, 5), 'Sporting Goods': (100, 250), 'Home': (100, 300)
    }
    sub_price_adjustment = {
        'Topwear': (5, 50), 'Watches': (30, 150), 'Shoes': (20, 100),
        'Bags': (10, 70), 'Fragrance': (5, 80), 'Jewellery': (10, 200)
    }
    article_price_range = {
        'Shirts': (15, 70), 'Watches': (50, 300), 'Sports Shoes': (40, 200),
        'Sunglasses': (30, 150), 'Perfumes': (25, 120)
    }

    # Lấy khoảng giá từ masterCategory
    base_min, base_max = master_price_range.get(masterCategory, (10, 100))

    # Điều chỉnh theo subCategory
    adj_min, adj_max = sub_price_adjustment.get(subCategory, (0, 0))

    # Điều chỉnh theo articleType
    art_min, art_max = article_price_range.get(articleType, (base_min + adj_min, base_max + adj_max))

    # Điều chỉnh theo năm
    if year < 2015:
        discount = 0.3
    elif year < 2018:
        discount = 0.15
    else:
        discount = 0

    final_min = art_min * (1 - discount)
    final_max = art_max * (1 - discount)

    return round(np.random.uniform(final_min, final_max) * 2500, 0)

In [ ]:
def add_price_column(example):
    example["price_in_vnd"] = assign_price(
        example["masterCategory"],
        example["subCategory"],
        example["articleType"],
        example["year"],
        example["season"],
        example["usage"]
    )
    return example


In [ ]:
def has_null(example):
    # Kiểm tra xem có giá trị nào trong example bị None không
    return any(val is None for val in example.values())

missing_null = train_dataset.filter(has_null)
print(f"Số record có giá trị null: {len(missing_null)}")

In [ ]:
dataset_dict["train"] = dataset_dict["train"].map(add_price_column)
# Split the training data into train and test (let's say 10% for the test set)
train_test_split = dataset_dict['train'].train_test_split(test_size=0.1, seed = 42)
# Further split the training set to get a validation set (e.g., 10% of the training set)
train_val_split = train_test_split['train'].train_test_split(test_size=0.1, seed = 42)

In [ ]:
# Combine the splits into a new DatasetDict
final_dataset = {
    'train': train_val_split['train'],
    'val': train_val_split['test'],
    'test': train_test_split['test']
}

final_dataset

In [ ]:
train_dataset = final_dataset["train"]
val_dataset = final_dataset["val"]
test_dataset = final_dataset["test"]

In [ ]:
train_dataset.features

In [ ]:
from datasets.features import Image
features = Features({
    'id': Value('int64'),
    'gender': Value('string'),
    'masterCategory': Value('string'),
    'subCategory': Value('string'),
    'articleType': Value('string'),
    'baseColour': Value('string'),
    'season': Value('string'),
    'year': Value('int64'),
    'usage': Value('string'),
    'productDisplayName': Value('string'),
    'image': Image(),
    'price_in_vnd': Value('int64')
})


In [ ]:
# Chuyển đổi dataset với features mới
train_dataset = train_dataset.cast(features)
val_dataset = val_dataset.cast(features)
test_dataset = test_dataset.cast(features)

## 2.1 Define Length in ProductDisplayName

In [ ]:
# df= df.drop(columns= 'token_length')
# df.to_csv(r'/home/ailab1/student_share/Thanh/GPU/Data/df_train.csv', index = False)

In [ ]:
df = pd.read_csv(r'/home/ailab1/student_share/Thanh/GPU/Data/df_train.csv')
df['token_length'] = df['productDisplayName'].apply(lambda x: len(tokenizer.tokenize(x)))


In [ ]:
# Kiểm tra phân phối độ dài token
print(df['token_length'].describe())
# Vẽ histogram độ dài tokens
plt.hist(df['token_length'], bins=30, color='blue', alpha=0.7)
plt.xlabel("Số lượng tokens")
plt.ylabel("Số dòng dữ liệu")
plt.title("Phân phối độ dài token")
plt.show()

In [ ]:
lengths = [len(tokenizer.tokenize(text)) for text in df["productDisplayName"]]
print(f"Độ dài trung bình: {np.mean(lengths)}")
print(f"Độ dài lớn nhất: {np.max(lengths)}")
print(f"90% mô tả có độ dài dưới: {np.percentile(lengths, 90)} tokens")
print(f"95% mô tả có độ dài dưới: {np.percentile(lengths, 95)} tokens")

## 2.3 Label Encoder

In [ ]:
def extract_brand(name, gender):
    # Chuyển đổi sang chữ thường để tìm kiếm không phân biệt hoa thường
    name_lower = name.lower()
    gender_lower = gender.lower()
    # Tách các từ của productDisplayName
    tokens = name.split(" ")
    tokens_lower = name_lower.split(" ")
    if gender_lower in tokens_lower:
        idx = tokens_lower.index(gender_lower)
        brand_tokens = tokens[:idx]  # Lấy tất cả từ trước từ gender
        # Tùy chỉnh: Nếu muốn loại bỏ từ "by" nếu nằm giữa, có thể xử lý thêm
        brand = " ".join(brand_tokens).replace("by", "").strip()
        return brand
    else:
        return tokens[0]

In [ ]:
def build_brand_mapping(dataset):
    # Duyệt qua từng productDisplayName và sử dụng cột gender tương ứng
    brands = []
    for name, gender in zip(dataset["productDisplayName"], dataset["gender"]):
        brands.append(extract_brand(name, gender))
    unique_brands = sorted(list(set(brands)))
    mapping = {brand: idx for idx, brand in enumerate(unique_brands)}
    inv_mapping = {idx: brand for brand, idx in mapping.items()}
    return mapping, inv_mapping

In [ ]:
# Hàm xây dựng mapping cho một cột
def build_mapping(dataset, col):
    unique_labels = dataset.unique(col)
    unique_labels.sort()  # Sắp xếp để nhất quán (tùy chọn)
    mapping = {label: idx for idx, label in enumerate(unique_labels)}
    inv_mapping = {idx: label for label, idx in mapping.items()}
    return mapping, inv_mapping

In [ ]:
# Ví dụ với các cột danh mục:
gender_map, inv_gender_map = build_mapping(train_dataset_org, "gender")
master_category_map, inv_master_category_map = build_mapping(train_dataset_org, "masterCategory")
sub_category_map, inv_sub_category_map = build_mapping(train_dataset_org, "subCategory")
article_type_map, inv_article_type_map = build_mapping(train_dataset_org, "articleType")
base_colour_map, inv_base_colour_map = build_mapping(train_dataset_org, "baseColour")
usage_map, inv_usage_map = build_mapping(train_dataset_org, "usage")
brand_map, inv_brand_map = build_brand_mapping(train_dataset_org)

# Kiểm tra các mapping:
print("Gender mapping:", gender_map)
print("Master Category mapping:", master_category_map)
print("Sub Category mapping:", sub_category_map)
print("Article Type mapping:", article_type_map)
print("Base Colour mapping:", base_colour_map)
print("Usage mapping:", usage_map)
print("Brand mapping:", brand_map)

In [ ]:
def update_categorical(example):
    # Cập nhật các cột theo mapping đã xây dựng

    example["masterCategory"] = master_category_map[example["masterCategory"]]
    example["subCategory"] = sub_category_map[example["subCategory"]]
    example["articleType"] = article_type_map[example["articleType"]]
    example["baseColour"] = base_colour_map[example["baseColour"]]
    example["usage"] = usage_map[example["usage"]]

    # Nếu dataset của bạn vẫn có cột "productDisplayName",
    # ta có thể trích xuất brand từ đó và chuyển sang giá trị số theo mapping:
    extracted = extract_brand(example["productDisplayName"], example["gender"])
    # Lưu ý: nếu không tìm thấy, extract_brand sẽ trả về token đầu tiên
    # và mapping brand có thể đã được xây dựng từ dữ liệu ban đầu.
    example["brand"] = brand_map.get(extracted, -1)  # Nếu không có, đặt giá trị -1 (hoặc giá trị mặc định)
    example["gender"] = gender_map[example["gender"]]
    return example

train_dataset = train_dataset.map(update_categorical)
val_dataset = val_dataset.map(update_categorical)
test_dataset = test_dataset.map(update_categorical)

In [ ]:
# Tạo MinMaxScaler
min_max_scaler = MinMaxScaler()

# Fit trên train dataset trước
scaling_values_train = np.array([train_dataset[col] for col in ["year", "price_in_vnd"]]).T
scaled_values_train = min_max_scaler.fit_transform(scaling_values_train)

# Áp dụng MinMaxScaler đã fit cho val & test
def scale_dataset(dataset, min_max_scaler):
    scaling_values = np.array([dataset[col] for col in ["year", "price_in_vnd"]]).T
    scaled_values = min_max_scaler.transform(scaling_values)  # Chỉ transform, không fit lại
    return dataset.map(lambda x, idx: {'year': scaled_values[idx][0], 'price_in_vnd': scaled_values[idx][1]}, with_indices=True)

train_dataset = scale_dataset(train_dataset, min_max_scaler)
val_dataset = scale_dataset(val_dataset, min_max_scaler)
test_dataset = scale_dataset(test_dataset, min_max_scaler)

In [ ]:
season_mapping = {'Spring': 0, 'Summer': 1, 'Fall': 2, 'Winter': 3}

train_dataset = train_dataset.map(lambda x: {'season': season_mapping[x['season']]})
val_dataset = val_dataset.map(lambda x: {'season': season_mapping[x['season']]})
test_dataset = test_dataset.map(lambda x: {'season': season_mapping[x['season']]})

In [ ]:
for col in ["gender", "masterCategory", "subCategory", "articleType", "baseColour", "season", "usage", "brand"]:
    unique_values = train_dataset.unique(col)
    print(f"Số lượng giá trị unique trong cột '{col}': {len(unique_values)}")
    print(f"Giá trị unique: {unique_values}")
    print("=" * 50)

## 2.4 BERT Tokenizer for Transformer

In [ ]:
def tokenize_function(example):
    return tokenizer(example["productDisplayName"], padding="max_length", truncation=True, max_length=13)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)
# Loại bỏ cột productDisplayName sau khi tokenized
train_dataset = train_dataset.remove_columns(["productDisplayName"])
val_dataset = val_dataset.remove_columns(["productDisplayName"])
test_dataset = test_dataset.remove_columns(["productDisplayName"])
train_dataset.features

## 2.5 Preprocess Image

In [ ]:
train_dataset[0]
import psutil

# Dung lượng RAM khả dụng và đã sử dụng
total_memory = psutil.virtual_memory().total / (1024**3)  # GB
used_memory = psutil.virtual_memory().used / (1024**3)  # GB
available_memory = psutil.virtual_memory().available / (1024**3)  # GB

print(f"🖥️ Tổng RAM: {total_memory:.2f} GB")
print(f"🚀 RAM đang dùng: {used_memory:.2f} GB")
print(f"🔄 RAM khả dụng: {available_memory:.2f} GB")

In [ ]:
import torch

if torch.cuda.is_available():
    gpu_memory_total = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    gpu_memory_used = torch.cuda.memory_allocated(0) / (1024**3)
    gpu_memory_free = gpu_memory_total - gpu_memory_used

    print(f"🖥️ Tổng bộ nhớ GPU: {gpu_memory_total:.2f} GB")
    print(f"🚀 GPU đang dùng: {gpu_memory_used:.2f} GB")
    print(f"🔄 GPU còn trống: {gpu_memory_free:.2f} GB")
else:
    print("❌ Không có GPU!")

In [ ]:
from torchvision import transforms
from PIL import Image
# Kiểm tra và chọn GPU nếu có
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load feature extractor của ViT
from transformers import ViTFeatureExtractor
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")

# Tạo transform chạy trên GPU
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std),
])

# Hàm xử lý ảnh trên GPU
def preprocess_function(batch):
    processed_images = []

    for img in batch["image"]:
        if isinstance(img, str):  # Nếu là đường dẫn, mở ảnh
            pil_img = Image.open(img).convert("RGB")
        else:
            pil_img = img.convert("RGB")  # Đảm bảo là RGB

        # Chuyển ảnh thành tensor và đẩy lên GPU
        tensor_img = transform(pil_img).to(device)
        processed_images.append(tensor_img)

    # Chuyển list thành tensor batch trên GPU
    batch["pixel_values"] = torch.stack(processed_images)

    return batch

# Áp dụng preprocessing trên GPU
train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    batch_size=32,
    num_proc=1,  # Giữ mức thấp vì đã chạy trên GPU
    remove_columns=["image"]
).with_format("torch")  # Chuyển dataset sang định dạng Torch

In [ ]:
# Lấy một batch từ dataset để kiểm tra
example_batch = next(iter(train_dataset))

# Chuyển từng ảnh trong `pixel_values` thành tensor
processed_pixel_values = [torch.tensor(img, dtype=torch.float32) for img in example_batch["pixel_values"]]

# Chuyển list thành batch tensor
pixel_values = torch.stack(processed_pixel_values)

# Kiểm tra kết quả
print("Pixel Values Shape:", pixel_values.shape)  # 🚀 Kỳ vọng: [batch_size, 3, 224, 224]
print("Pixel Values dtype:", pixel_values.dtype)  # 🚀 Kỳ vọng: torch.float32


In [ ]:
val_dataset = val_dataset.map(
    preprocess_function,
    batched=True,
    batch_size=32,
    num_proc=1,
    remove_columns=["image"]
).with_format("torch")

In [ ]:
test_dataset = test_dataset.map(
    preprocess_function,
    batched=True,
    batch_size=32,
    num_proc=1,
    remove_columns=["image"]
).with_format("torch")

# III.  Fine-tuning Model (ViT)

In [ ]:
available_ram = psutil.virtual_memory().available
cpu_count = psutil.cpu_count(logical=True)
print(f"Available RAM: {available_ram / (1024**3):.2f} GB")
print(f"CPU count: {cpu_count}")

In [ ]:
from transformers import ViTModel
# --- 1. Load mô hình ViT pre-trained ở chế độ 8-bit ---
# Sử dụng bitsandbytes để tải model ở định dạng 8-bit nhằm tiết kiệm bộ nhớ GPU
vit_model = ViTModel.from_pretrained(
    "google/vit-base-patch16-224",
    # load_in_8bit=True,       # 8-bit quantization
    device_map={"": 0}        # Tự động phân bổ mô hình trên GPU
)

## 3.1 Xây dựng Mô Hình ViT Tùy Chỉnh (Multi-Task Learning)

In [ ]:
import torch.nn.functional as F

class CustomViTHierarchical(nn.Module):
    def __init__(self,
                 vit_model,
                 num_gender=5,
                 num_master=7,
                 num_usage=8,
                 num_sub=45,
                 num_article=142,
                 num_base=46,
                 num_brand=820):
        super(CustomViTHierarchical, self).__init__()
        self.vit = vit_model
        hidden_size = self.vit.config.hidden_size

        # Level 1
        self.head1_gender = nn.Linear(hidden_size, num_gender)
        self.head1_master = nn.Linear(hidden_size, num_master)
        self.head1_usage  = nn.Linear(hidden_size, num_usage)
        self.level1_dim = num_gender + num_master + num_usage

        # Level 2
        self.head2 = nn.Sequential(
            nn.Linear(hidden_size + self.level1_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_sub)
        )
        self.level2_dim = self.level1_dim + num_sub

        # Level 3
        self.head3 = nn.Sequential(
            nn.Linear(hidden_size + self.level2_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_article)
        )
        self.level3_dim = self.level2_dim + num_article

        # Level 4
        self.head4 = nn.Sequential(
            nn.Linear(hidden_size + self.level3_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_base + num_brand)
        )
        self.num_base = num_base
        self.num_brand = num_brand

    def forward(self, pixel_values, labels=None, **kwargs):
        outputs = self.vit(pixel_values=pixel_values)
        cls_output = outputs.last_hidden_state[:, 0, :]  # (batch, hidden_size)

        # Level 1
        pred_gender = self.head1_gender(cls_output)
        pred_master = self.head1_master(cls_output)
        pred_usage  = self.head1_usage(cls_output)
        level1_pred = torch.cat([pred_gender, pred_master, pred_usage], dim=1)

        # Level 2
        level2_input = torch.cat([cls_output, level1_pred], dim=1)
        pred_sub = self.head2(level2_input)
        level2_pred = torch.cat([level1_pred, pred_sub], dim=1)

        # Level 3
        level3_input = torch.cat([cls_output, level2_pred], dim=1)
        pred_article = self.head3(level3_input)
        level3_pred = torch.cat([level2_pred, pred_article], dim=1)

        # Level 4
        level4_input = torch.cat([cls_output, level3_pred], dim=1)
        pred_base_brand = self.head4(level4_input)
        pred_base = pred_base_brand[:, :self.num_base]
        pred_brand = pred_base_brand[:, self.num_base:]

        outputs_dict = {
            "gender": pred_gender,
            "master": pred_master,
            "usage": pred_usage,
            "sub": pred_sub,
            "article": pred_article,
            "base": pred_base,
            "brand": pred_brand
        }

        # Nếu labels được cung cấp, tính loss cho từng task và cộng dồn
        if labels is not None:
            loss = 0
            for key in labels:
                # Giả sử các nhãn trong labels có cùng các key: "gender", "master", ...
                loss += F.cross_entropy(outputs_dict[key], labels[key])
            outputs_dict["loss"] = loss

        return outputs_dict

In [ ]:
model = CustomViTHierarchical(vit_model)
# Freeze tất cả tham số trong backbone ViT
for param in vit_model.parameters():
    param.requires_grad = False


In [ ]:
print("Danh sách các module chứa 'attn' hoặc 'attention':")
for name, module in model.named_modules():
    if 'attn' in name or 'attention' in name:
        print(name)
        for sub_name, sub_module in module.named_modules():
            print(f"  - {sub_name}")
from peft import LoraConfig, get_peft_model


In [ ]:
# --- 4. Cấu hình và chèn LoRA vào mô hình ---
# Chúng ta áp dụng LoRA vào các projection layers của phần attention trong backbone ViT.
# Vì model của bạn có ViT được lưu trong attribute 'vit', nên tên các module sẽ có tiền tố 'vit.'.
lora_config = LoraConfig(
    r=16,                # Rank của ma trận low-rank
    lora_alpha=32,       # Hệ số scaling cho LoRA
    target_modules=[
        "query",
        "key",
        "value",
        "dense"
    ],
    lora_dropout=0.05,   # Tỉ lệ dropout cho LoRA
    bias="none",         # Không cập nhật bias
    task_type="FEATURE_EXTRACTION"  # Loại nhiệm vụ (điều chỉnh nếu cần)
)
# Chèn LoRA vào mô hình (freeze hầu hết tham số, chỉ cập nhật các tham số LoRA)
model = get_peft_model(model, lora_config)

In [ ]:
# --- 5. In số lượng tham số trainable để xác nhận ---
def print_trainable_parameters(model):
    trainable_params = 0
    total_params = 0
    for name, param in model.named_parameters():
        total_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"Trainable params: {trainable_params} || Total params: {total_params} || Trainable%: {100 * trainable_params / total_params:.2f}%")

print_trainable_parameters(model)

In [ ]:
import torch
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    metrics = {}
    total_loss = 0.0
    num_tasks = 0

    # Giả sử các task có các key: "gender", "master", "usage", "sub", "article", "base", "brand"
    for key in labels:
        # Chuyển đổi nếu cần (đảm bảo là tensor)
        task_logits = torch.tensor(logits[key]) if not torch.is_tensor(logits[key]) else logits[key]
        task_labels = torch.tensor(labels[key]) if not torch.is_tensor(labels[key]) else labels[key]

        # Tính loss cho task hiện tại
        loss_val = F.cross_entropy(task_logits, task_labels, reduction="mean")
        total_loss += loss_val.item()
        num_tasks += 1

        # Tính accuracy và f1
        pred_ids = torch.argmax(task_logits, dim=-1).cpu().numpy()
        true_ids = task_labels.cpu().numpy()
        acc = accuracy_score(true_ids, pred_ids)
        f1 = f1_score(true_ids, pred_ids, average='weighted')

        metrics[f"{key}_accuracy"] = acc
        metrics[f"{key}_f1"] = f1

    # Tính loss trung bình cho tất cả các task và trả về dưới key "eval_loss"
    metrics["eval_loss"] = total_loss / num_tasks if num_tasks > 0 else 0.0

    # Tính trung bình các chỉ số accuracy và f1 nếu cần
    acc_values = [v for k, v in metrics.items() if k.endswith('_accuracy')]
    f1_values = [v for k, v in metrics.items() if k.endswith('_f1')]
    metrics["eval_average_accuracy"] = sum(acc_values) / len(acc_values) if acc_values else 0.0
    metrics["eval_average_f1"] = sum(f1_values) / len(f1_values) if f1_values else 0.0

    return metrics

In [ ]:
small_test_dataset = test_dataset.select(range(10))
predictions = trainer.predict(small_test_dataset)
print("Predictions:", predictions.metrics)

In [ ]:
small_eval_dataset = val_dataset.select(range(10))  # chọn 10 mẫu đầu tiên
eval_metrics = trainer.evaluate(eval_dataset=small_eval_dataset)
print("Eval metrics:", eval_metrics)

In [ ]:
import torch
import torch.nn.functional as F
from transformers import Trainer

def custom_compute_loss(model, inputs, num_items_in_batch):
    # Lấy nhãn từ inputs
    labels = {
        "gender": inputs["gender"],
        "master": inputs["master"],
        "usage": inputs["usage"],
        "sub": inputs["sub"],
        "article": inputs["article"],
        "base": inputs["base"],
        "brand": inputs["brand"],
    }

    # Gọi model với các inputs cần thiết
    outputs = model(
        pixel_values=inputs["pixel_values"],
        input_ids=inputs["input_ids"],
        token_type_ids=inputs["token_type_ids"],
        attention_mask=inputs["attention_mask"]
    )

    # Tính loss cho từng task (sử dụng CrossEntropyLoss)
    loss = 0
    for key in labels:
        loss += F.cross_entropy(outputs[key], labels[key])

    return loss

In [ ]:
import torch

def data_collator(features):
    # Vì các trường này đã ở dạng tensor, nên dùng torch.stack() thay vì torch.tensor()
    input_ids = torch.stack([f["input_ids"] for f in features])
    token_type_ids = torch.stack([f["token_type_ids"] for f in features])
    attention_mask = torch.stack([f["attention_mask"] for f in features])

    # Ghép batch của ảnh
    pixel_values = torch.stack([f["pixel_values"].clone().detach().float() for f in features])

    # Các nhãn thường là kiểu số nguyên nên dùng torch.tensor() là ổn
    labels = {
        "gender": torch.tensor([f["gender"] for f in features], dtype=torch.long),
        "master": torch.tensor([f["masterCategory"] for f in features], dtype=torch.long),
        "usage": torch.tensor([f["usage"] for f in features], dtype=torch.long),
        "sub": torch.tensor([f["subCategory"] for f in features], dtype=torch.long),
        "article": torch.tensor([f["articleType"] for f in features], dtype=torch.long),
        "base": torch.tensor([f["baseColour"] for f in features], dtype=torch.long),
        "brand": torch.tensor([f["brand"] for f in features], dtype=torch.long),
    }

    return {
        "pixel_values": pixel_values,
        "input_ids": input_ids,
        "token_type_ids": token_type_ids,
        "attention_mask": attention_mask,
        **labels,
    }


In [ ]:
# Lấy một số mẫu từ train_dataset (ví dụ 5 mẫu)
sample_features = [train_dataset[i] for i in range(5)]

In [ ]:
# Gọi hàm data_collator trên các mẫu đó
batch = data_collator(sample_features)

In [ ]:
# In thông tin của từng key trong batch
print("Các key có trong batch:", list(batch.keys()))
print("Pixel Values shape:", batch["pixel_values"].shape)
print("Input IDs shape:", batch["input_ids"].shape)
print("Token Type IDs shape:", batch["token_type_ids"].shape)
print("Attention Mask shape:", batch["attention_mask"].shape)

In [ ]:
# In shape của các nhãn
for label_key in ["gender", "master", "usage", "sub", "article", "base", "brand"]:
    print(f"{label_key} shape:", batch[label_key].shape)


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./vit_hierarchical",
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=1,
    save_strategy="steps",
    save_steps=1,  # Lưu mỗi epoch thay vì mỗi 500 step
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    fp16=True,  # ⚡️ Mixed Precision (Tăng tốc GPU)
    load_best_model_at_end=True,
    dataloader_num_workers=0,  # 🔥 Tắt multi-processing tránh lỗi CUDA fork
    # metric_for_best_model="average_f1",
    remove_unused_columns=False,  # ⚠️ Quan trọng: tránh lỗi khi có nhiều feature
    logging_first_step=True,
    dataloader_pin_memory=False,
)

In [ ]:
from transformers import Trainer

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # Lấy num_items_in_batch từ kwargs nếu có, ngược lại lấy batch size
        num_items_in_batch = kwargs.get("num_items_in_batch", len(inputs["input_ids"]))
        # Tính loss bằng hàm custom_compute_loss mà bạn đã định nghĩa
        loss = custom_compute_loss(model, inputs, num_items_in_batch=num_items_in_batch)
        if return_outputs:
            return loss, None
        return loss


In [ ]:
from transformers import ViTImageProcessor

# model_name = "google/vit-large-patch16-224"
processor = ViTImageProcessor.from_pretrained("google/vit-large-patch16-224")
processor
# Khởi tạo CustomTrainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    tokenizer=processor,

)

In [ ]:
# Start training
trainer.train()

In [ ]:
outputs = trainer.predict(test_dataset)
print(outputs.metrics)

In [ ]:
# Lưu model và cấu hình
model.save_pretrained("./my_finetuned_vit")

In [ ]:
# Nếu có tokenizer (ví dụ: BERT tokenizer)
tokenizer.save_pretrained("./my_finetuned_vit")

from transformers import ViTModel, BertTokenizer

model = ViTModel.from_pretrained("./my_finetuned_vit")
tokenizer = BertTokenizer.from_pretrained("./my_finetuned_vit")
